In [10]:
import iris

In [26]:
import importlib
importlib.reload(iris)

<module 'Iris' from 'E:\\REPOS\\int_sys\\swarm\\Iris.py'>

In [12]:
nn = NeuralNetwork.IrisNN()

z = nn.W1
print(nn.softmax(z))

from sklearn.utils.extmath import softmax

print()

print(softmax(z))

[[ 0.1178485   0.16079573  0.17878259  0.11053541  0.21569607  0.21634171]
 [ 0.22102079  0.18365881  0.22444359  0.10713691  0.1671221   0.0966178 ]
 [ 0.1063425   0.09038538  0.23754115  0.14542505  0.22906536  0.19124056]
 [ 0.0919226   0.19877088  0.20524326  0.23353816  0.09636597  0.17415913]]

[[ 0.1178485   0.16079573  0.17878259  0.11053541  0.21569607  0.21634171]
 [ 0.22102079  0.18365881  0.22444359  0.10713691  0.1671221   0.0966178 ]
 [ 0.1063425   0.09038538  0.23754115  0.14542505  0.22906536  0.19124056]
 [ 0.0919226   0.19877088  0.20524326  0.23353816  0.09636597  0.17415913]]


In [14]:
from sklearn import datasets

df = datasets.load_iris()
print(df.target)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [29]:
df = Iris.IrisDF()

df.subset()
print(len(df.X_train))

24
